In [4]:
using DataFrames;
using Distributions;

In [54]:
df = readtable("/Users/nwelch/prelim/data/plantDataTest.csv");
dst = readtable("/Users/nwelch/prelim/data/plantDistanceTest.csv");

# Open Issues: 
* Note that the last log sums in llRatio_tau have reversed signs than what is shown in the appendix. The Appendix appears to have the wrong sign. 

# Initial Parameter Values

In [55]:
#priors
mu_a=0.7; mu_b=0.004;
theta_a=0.8; theta_b=10;
sigma_a=0.5; sigma_b=100;

# initial/test values
u = mu_a*mu_b
th = theta_a*theta_b;
s = sigma_a*sigma_b;
t = [14., 10., 6.];
tl = maximum(t);

# Function Definitions

## log  Lambda

**Scrubbed and equivalent to R-unit test results.**

In [56]:
function getLogLambda(tau, theta, mu, sigma, dst, Tlast=30.)
    tauLessTIndex = find(x-> x<=Tlast, tau)
    gaussian = Normal(mu, sigma)

    sumLog = 0
    for i = tauLessTIndex
        sumTauJLessTauI = 0
        tauJLessTauIIndex = find(x-> x < tau[i], tau)
        for j = tauJLessTauIIndex
            sumTauJLessTauI = sumTauJLessTauI + pdf(gaussian, dst[i,j])
        end
        sumLog = sumLog + log(mu + theta*sumTauJLessTauI)
    end
    sumLog
end

getLogLambda (generic function with 2 methods)

In [57]:
getLogLambda(t, th, u, s, dst, tl)

-10.631796271049293

R Value: -10.623570479769231

## $\mu$ log-likelihood Ratio

**Scrubbed and equivalent to R-unit test results.**

In [9]:
function llRatio_mu(mu, muStar, theta, sigma, tau, dst; 
                        Tlast=30.0, mu_a=0.7, mu_b=0.004)
    muPrior = Gamma(mu_a, mu_b)

    tauLessTIndex = find(x-> x<=Tlast, tau)
    tauGreaterTIndex = find(x-> x>Tlast, tau)

    dmu = mu - muStar

    dllTerm1 = sum(tau[tauLessTIndex]*dmu)
    dllTerm2 = length(tauGreaterTIndex) * Tlast * dmu

    dllTerm3a = getLogLambda(tau, theta, muStar, sigma, dst, Tlast)
    dllTerm3b = getLogLambda(tau, theta, mu, sigma, dst, Tlast)
    dllTerm3 = dllTerm3a - dllTerm3b

    logPriorStar = log(pdf(muPrior, muStar))
    logPrior = log(pdf(muPrior, mu))

    dll = dllTerm1 + dllTerm2 + dllTerm3 + logPriorStar - logPrior
    dll
end

llRatio_mu (generic function with 1 method)

In [58]:
llRatio_mu(u, 0.01, th, s, t, dst, Tlast=tl)

-0.967916877573046

R Value: -0.9685811338229993

## $\sigma$ log-likelihood Ratio

**Scrubbed and equivalent to R-unit test results.**

In [12]:
function getSumLessT_sigma(sigma, sigmaStar, gaussian_s, gaussianStar_s, mu,
                                  tau, theta, dst, Tlast=30.0)
    tauLessTIndex = find(x -> x<=Tlast, tau)

    sumLessT = 0
    for i = tauLessTIndex
        sum_ij = 0
        tauJLessTauIIndex = find(x-> x<tau[i], tau)
        for j=tauJLessTauIIndex
            tmp = (tau[i] - tau[j])*(pdf(gaussian_s, dst[i,j]) -
                                   pdf(gaussianStar_s, dst[i,j]))
            sum_ij = sum_ij + tmp
        end
        sumLessT = sumLessT + theta*sum_ij
    end
    sumLessT
end

getSumLessT_sigma (generic function with 2 methods)

In [59]:
sStar = 1.0
gaussian_s = Normal(u, s);
gaussianStar_s = Normal(u, sStar);
getSumLessT_sigma(s, sStar, gaussian_s, gaussianStar_s, u, t, th, dst, tl)

-6.748482930483687

R Value: -37.07198157467801

In [18]:
function getSumGreaterT_sigma(sigma, sigmaStar, gaussian_s, gaussianStar_s,
                                     mu, tau, theta, dst, Tlast=30.)

    tauGreaterTIndex = find(x -> x>Tlast, tau)

    sumGreaterT = 0
    for i=tauGreaterTIndex
        sum_ij = 0
        tauJLessTauIIndex = find(x-> x<tau[i], tau)
        for j=tauJLessTauIIndex
            sum_ij = sum_ij +
                (Tlast - tau[j])*(pdf(gaussian_s, dst[i,j]) -
                                  pdf(gaussianStar_s, dst[i,j]))
        end
        sumGreaterT = sumGreaterT + theta*sum_ij
    end
    sumGreaterT
end

getSumGreaterT_sigma (generic function with 2 methods)

In [60]:
sStar = 100.
gaussian_s = Normal(u, s);
gaussianStar_s = Normal(u, sStar);
getSumGreaterT_sigma(s, sStar, gaussian_s, gaussianStar_s, u, t, th, dst, 10.)

0.126060878919472

R Value: 0.12761710295072787

In [20]:
function llRatio_sigma(sigma, sigmaStar, gaussian_s, gaussianStar_s,  mu,
                              tau, theta, dst; 
                              Tlast=30.0, sigma_a=0.5, sigma_b=100.)

    sigmaPrior = Gamma(sigma_a, sigma_b)

    dllTerm1 = getSumLessT_sigma(sigma, sigmaStar, gaussian_s, gaussianStar_s,
                                        mu, tau, theta, dst)

    dllTerm2 = getSumGreaterT_sigma(sigma, sigmaStar, gaussian_s,
                                           gaussianStar_s,
                                           mu, tau, theta, dst)

    dllTerm3 = getLogLambda(tau, theta, mu, sigmaStar, dst, Tlast) -
        getLogLambda(tau, theta, mu, sigma, dst, Tlast)

    logPriorStar = log(pdf(sigmaPrior, sigmaStar))
    logPrior = log(pdf(sigmaPrior, sigma))

    dll = dllTerm1 + dllTerm2 + dllTerm3 + logPriorStar - logPrior

    dll
end

llRatio_sigma (generic function with 1 method)

In [61]:
sStar = 1.0
gaussian_s = Normal(u, s);
gaussianStar_s = Normal(u, sStar);
llRatio_sigma(s, sStar, gaussian_s, gaussianStar_s, u, t, th, dst, Tlast=tl)

-4.757907999644139

R Value: -29.676456732636904 (*before log(prior) correction*)

# $\theta$ log-likelihood Ratio

**Scrubbed and equivalent to R-unit test results.**

In [29]:
gaussian_t = Normal(u, s)

Distributions.Normal{Float64}(μ=0.0028, σ=50.0)

In [26]:
function getSumLessT_theta(theta, thetaStar, mu, tau, sigma, gaussian_t, dst,
                                  Tlast=30.0)

    tauLessTIndex = find(x -> x<=Tlast, tau)

    sumLessT = 0
    for i = tauLessTIndex
        sum_ij = 0
        tauJLessTauIIndex = find(x-> x<tau[i], tau)
        for j = tauJLessTauIIndex
            sum_ij = sum_ij +
                (tau[i] - tau[j])*pdf(gaussian_t, dst[i,j])
        end
        sumLessT = sumLessT + (theta - thetaStar)*sum_ij
    end
    sumLessT
end

getSumLessT_theta (generic function with 2 methods)

In [27]:
getSumLessT_theta(th, 1., u, t, s, gaussian_t, dst, tl)

0.893519762891437

R Value: 0.893519762891437

In [28]:
function getSumGreaterT_theta(theta, thetaStar, mu, tau, sigma, gaussian_t,
                                     dst, Tlast=30.0)

    tauGreaterTIndex = find(x -> x>Tlast, tau)

    sumGreaterT = 0
    for i=tauGreaterTIndex
        sum_ij = 0
        tauJLessTauIIndex = find(x-> x<tau[i], tau)
        for j=tauJLessTauIIndex
            sum_ij = sum_ij +
                (Tlast - tau[j])*pdf(gaussian_t, dst[i,j])
        end
        sumGreaterT = sumGreaterT + (theta - thetaStar)*sum_ij
    end
    sumGreaterT
end

getSumGreaterT_theta (generic function with 2 methods)

In [30]:
getSumGreaterT_theta(th, 1., u, t, s, gaussian_t, dst, 10.)

0.2233632497737195

R Value: 0.2233632497737195

In [31]:
function llRatio_theta(theta, thetaStar, mu, tau, sigma, gaussian_t, dst; 
                         Tlast=30.0, theta_a=0.8, theta_b=10.0)

    thetaPrior = Gamma(theta_a, theta_b)
    dllTerm1 = getSumLessT_theta(theta, thetaStar, mu, tau, sigma, gaussian_t, 
                                        dst, Tlast)

    dllTerm2 = getSumGreaterT_theta(theta, thetaStar, mu, tau, sigma,
                                           gaussian_t,  dst, Tlast)

    dllTerm3 = getLogLambda(tau, thetaStar, mu, sigma, dst, Tlast) -
                    getLogLambda(tau, theta, mu, sigma, dst, Tlast)

    logPriorStar = log(pdf(thetaPrior, thetaStar))
    logPrior = log(pdf(thetaPrior, theta))

    dll = dllTerm1 + dllTerm2 + dllTerm3 + logPriorStar - logPrior
    dll
end

llRatio_theta (generic function with 1 method)

In [33]:
llRatio_theta(th, 1., u, t, s, gaussian_t, dst, Tlast=tl)

-1.7516207513257855

R Value: -1.7516207513257855

# $\tau$ log-likelihood Ratio

**Scrubbed and equivalent to R-unit test results.**

In [53]:
time_ns()

0x000012e2799d69be

In [35]:
gaussian_tau = Normal(u, s)

Distributions.Normal{Float64}(μ=0.0028, σ=50.0)

In [36]:
function getMinSum(iStar, tauiStar, mu, tau, theta, sigma, gaussian_tau, dst)
    ti = tau[iStar]
    minTau = min(tauiStar, ti)

    jLessMin = find(x -> x<minTau, tau)
    minSum = 0
    for j = jLessMin
        minSum = minSum + pdf(gaussian_tau, dst[iStar,j])
    end
    minSum = (ti - tauiStar)*theta*minSum
    minSum
end

getMinSum (generic function with 1 method)

In [37]:
getMinSum(1, t[1]+0.5, u, t, th, s, gaussian_tau, dst)

-0.06382284020653121

R Value: -0.06382284020653121

In [39]:
function getMaxSum(iStar, tauiStar, mu, tau, theta, sigma, gaussian_tau, dst)
    ti = tau[iStar]
    maxTau = max(tauiStar, ti)

    jGreaterMax = find(x -> x>maxTau, tau)
    maxSum = 0
    for j = jGreaterMax
        maxSum = maxSum + pdf(gaussian_tau, dst[iStar,j])
    end
    maxSum = (tauiStar - ti)*theta*maxSum
    maxSum
end

getMaxSum (generic function with 1 method)

In [40]:
getMaxSum(2, t[2]-0.5, u, t, th, s, gaussian_tau, dst)

-0.031913804524571286

R Value: -0.031913804524571286

In [42]:
function getSumIntervalBelowTauStar(iStar, tauiStar, mu, tau, theta, sigma,
                                           gaussian_tau, dst)

    ti = tau[iStar]

    jInBetween = find(x -> ((x>ti) & (x<tauiStar)), tau)
    sumBelow = 0
    for j = jInBetween
        sumBelow = sumBelow +
            (2*tau[j] - ti - tauiStar)*pdf(gaussian_tau, dst[iStar,j])
    end
  sumBelow = theta*sumBelow
  sumBelow
end

getSumIntervalBelowTauStar (generic function with 1 method)

In [43]:
getSumIntervalBelowTauStar(2, 15., u, t, th, s, gaussian_tau, dst)

0.1914828271474277

R Value: 0.1914828271474277

In [46]:
function getSumIntervalAboveTauStar(iStar, tauiStar, mu, tau, theta, sigma,
                                           gaussian_tau, dst)
    ti = tau[iStar]

    jInBetween = find(x -> ((x>tauiStar) & (x<ti)), tau)
    sumAbove = 0
    for j = jInBetween
            sumAbove = sumAbove +
                (ti + tauiStar - 2*tau[j])*pdf(gaussian_tau, dst[iStar,j])
    end
    sumAbove = theta*sumAbove
    sumAbove
end

getSumIntervalAboveTauStar (generic function with 1 method)

In [47]:
getSumIntervalAboveTauStar(1, 8., u, t, th, s, gaussian_tau, dst)

0.12765521809828514

R Value: 0.12765521809828514

In [49]:
function llRatio_tau(tau, tauiStar, iStar, mu, theta, sigma, gaussian_tau,
                          dst; 
                          Tlast=30.0)
    N = length(tau)

    dllTerm1 = mu*(tau[iStar] - tauiStar)
    dllTerm2 = getMinSum(iStar, tauiStar, mu, tau, theta, sigma,
                                gaussian_tau, dst)
    dllTerm3 = getMaxSum(iStar, tauiStar, mu, tau, theta, sigma,
                                gaussian_tau, dst)
    dllTerm4 = getSumIntervalBelowTauStar(iStar, tauiStar, mu, tau,
                                          theta, sigma, gaussian_tau, dst)
    dllTerm5 = getSumIntervalAboveTauStar(iStar, tauiStar, mu, tau,
                                          theta, sigma, gaussian_tau, dst)

    tauStar = copy(tau)
    tauStar[iStar] = tauiStar
    dllTerm6 = getLogLambda(tauStar, theta, mu, sigma, dst, Tlast)
    dllTerm7 = getLogLambda(tau, theta, mu, sigma, dst, Tlast)

    dll = dllTerm1 + dllTerm2 + dllTerm3 + dllTerm4 + dllTerm5 +
        dllTerm6 - dllTerm7
    dll
end

llRatio_tau (generic function with 1 method)

In [51]:
llRatio_tau(t, t[1]-0.5, 1, u, th, s, gaussian_tau, dst,Tlast=tl)

0.06522284020653046

R Value: 0.06522284020653046